# Titanic: Survival Model (XGBoost Version)

Build and train an XGBoost model to predict survival on the Titanic using a [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410), and upload the model to Hugging Face.

Based on https://github.com/jamieoliver/titanic-model-2410.

## Plan
- [ ] Download [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410) from Hugging Face
- [ ] Prepare data for model
- [ ] Build and train initial model
- [ ] Tune model hyperparameters
- [ ] Test final model
- [ ] Upload model to Hugging Face